# MiniFlow
Tensorflow的简版

In [1]:
import numpy as np

## 基类

In [2]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # 一个用于存储对传入节点的引用，另一个用于存储对传出节点的引用。
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        # 对于每一个传入节点，其输出节点都包含本节点；
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        
        # 该节点的值
        self.value = None
        
    # 向前传播
    def forward(self):
        pass

## Input类

In [3]:
class Input(Node):
    # input子类没有输入值，初始化不用传参
    def __init__(self):
        Node.__init__(self)
    # 只有Input类的向前传播需要传参，别的类都是从inputNodes中取值
    def forward(self, value=None):
        if value is not None:
            self.value = value

## Add子类，实现加法

In [4]:
class Add(Node):
    def __init__(self,*inputs):
        Node.__init__(self, inputs)
    def forward(self):
        self.value = 0
        for node in self.inbound_nodes:
            self.value += node.value

## Linear

In [5]:
class Linear(Node):
    def __init__(self, X, W, B):
        self.__init__(self,[X,W,B])
    
    def forward(self):
        x_value = self.inbound_nodes[0].value
        w_value = self.inbound_nodes[1].value
        bias_value = self.inbound_nodes[2].value
        self.value = np.dot(x_value,w_value) + bias_value

## Sigmoid

In [6]:
class Sigmoid(Node):
    def __init__(self,node):
        Node.__init__(self, [node])
    
    def _sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    def forward(self):
        self.value = self._sigmoid(self.inbound_nodes[0].value)

## 均方误差

In [8]:
class MSE(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y,a])
        
    def forward(self):
        y = self.inbound_nodes[0].value.reshape(-1,1)
        a = self.inbound_nodes[1].value.reshape(-1,1)
        self.value = np.sum(np.square(y-a))/np.shape(y)[0]